# Connecting to PostgreSQL 

Example of connecting to PostgreSQL databases from Python using the [`psycopg2`](http://initd.org/psycopg/) package.

`psycopg2` is the most popular package for connecting to PostgreSQL databases.  There are frameworks, such as [`sqlalchemy`](https://www.sqlalchemy.org/), built on top of it that add additional functionality and ways of working with databases.  

You need to have installed `psycopg2` before running this notebook.

The packages for connecting to other types of SQL databases have a similar syntax and workflow.  They are governed by the [Python Database API Specification](https://www.python.org/dev/peps/pep-0249/).

In [1]:
import psycopg2

## Connect

Fill in the connection information below.

Note that in scripts that you write, especially those that you share, you probably don't want to hard code passwords into your file.  Alternatives are to read in an environment variable, read values from a configuration file that you don't share (just share a blank template), or include a step in your script or application to read the password from a command line argument or input from the user.

In [ ]:
conn = psycopg2.connect(dbname="", host="", 
                        user="", password="")

## Get a Cursor

You need a cursor to execute queries.

In [ ]:
cur = conn.cursor()

## Execute Statements

Using the cursor, you can then execute SQL statements.  By default, statements are part of a transaction and are not permanent until you commit (or you can rollback) the connection. 

In [ ]:
cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num float, name text);")

Insert some random data

In [ ]:
import random
from string import ascii_lowercase
for val in ascii_lowercase:
    cur.execute("INSERT INTO test (num, name) VALUES (%s, %s)", (random.random(), val))

Above, val will get quoted automatically when inserted into the statement.

Now, make sure to commit

In [ ]:
conn.commit() ## conn, not cur

## Getting Results

You fetch results back into Python.

In [ ]:
cur.execute("SELECT * FROM test;")
print(cur.fetchone()) ## get one result (you get tuples)

Once that first row is fetched, it's not the result set anymore.  If we pull more results, the first row won't be included.  This means also that if we re-run a fetch command, previously fetched results will be gone.

In [ ]:
for row in cur.fetchall(): ## fetch everything still in the result set
    print("ID: {}\nNUM: {};\nNAME: {}\n".format(*row)) # * unpacks the row tuple

Note that results are unnamed tuples.  We have to keep track ourselves of what value is in what position in the results.  We can get a list of column names if we need:

In [ ]:
colnames = [desc[0] for desc in cur.description]
print(colnames)

Or we could use `pandas` (see below).

## RISK: SQL Injection

If you're not careful, if you accept values from a user and use them directly in a database query, a malicious user can insert code to harm your database.  To avoid this, don't construct SQL queries as strings on your own.  Instead, use the built in functionality to put placeholders for values in the query, and then send a tuple with Python objects that the values will be read from.  The latter way, the values get sanitized.

For more details, see [`psycopg2` documentation](http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries).

![xkcd sql injection](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)
Source: https://xkcd.com/327/

In [ ]:
# BAD!!!!

myval = 3
cur.execute("select * from test where id="+str(myval)+";")
print(cur.fetchone())

In [ ]:
# CORRECT

myval = 3
cur.execute("select * from test where id=%s;", [myval]) ## use a list or tuple for values
print(cur.fetchone())

## Close Cursor and Connection

They will get closed when your script terminates, but it's good practice anyway.

In [ ]:
cur.close()
conn.close()

## Notes

There are methods for copying data to the database from a file and copying a table from the database to a file.  See the `psycopg2` documentation for details and examples.

# Pandas

You can read data directly into a `pandas` data frame too.

In [ ]:
import pandas as pd

In [ ]:
conn = psycopg2.connect(dbname="", host="", 
                        user="", password="")

In [ ]:
df = pd.read_sql_query("select * from test;", conn)
df

There are functions for creating tables from pandas data frames, but these currently only work when using SQLite unless you are also using `sqlalchemy`.

In [ ]:
conn.close()